# Accessing the profiling raw data

In [ ]:
import tslumen
import pandas as pd

In [ ]:
# patch it
_read_csv = pd.read_csv
pd.read_csv = lambda fpath, *args, **kwargs: _read_csv(fpath.split('/')[-1], *args, **kwargs)

In [ ]:
df = pd.read_csv(
    'https://datahub.io/core/gdp/r/gdp.csv', 
    parse_dates=[2], 
    date_parser=lambda dt: pd.to_datetime(f'{dt}-12-31'),
)
df = df[df['Country Code'].isin(['HIC', 'MIC', 'LIC'])]\
    .set_index(['Year', 'Country Code'])['Value'].unstack(1)
meta = {
    'frame': {
        'Description': 'Country, regional and world GDP in current US Dollars ($). Regional means collections of countries e.g. Europe & Central Asia.',
        'Source': '<a href="https://datahub.io/core/gold-prices">DataHub</a>',
    },
    'series': {
        'HIC': 'High income GDP in current USD',
        'MIC': 'Middle income GDP in current USD',
        'LIC': 'Low income GDP in current USD',
    }
}

In [ ]:
profile = tslumen.DefaultProfiler().profile(df)

The profiling results are stored in a `dataclass` containing summary information like name, start and end timestamps and errors:

In [ ]:
print(f'''\
name:         {profile.name}
start:        {profile.start}
end:          {profile.end}
config:       dictionary with {len(profile.result.config)} elements
success:      {profile.success}
warnings:     {profile.warnings}
exception:    {profile.exception}
exec_details: ''')
display(profile.result.exec_details)

Result is pretty printed when using an IPython environment like Jupyter.

In [ ]:
profile

The results of each profiling function are stored under the `frame` and `series` attributes, depending on whether the function works at DataFrame (looks at multiple series together) or Series (looks at a single time series) level.
  * `frame` is a flat dictionary indexed by the profiling function name
  * `series` is a 2-level deep dictionary, first indexed by the series (column) name, then by profiling function

In [ ]:
profile.result.frame.keys()

In [ ]:
profile.result.series.keys()

In [ ]:
profile.result.series['HIC'].keys()

In [ ]:
profile.result.frame['sz_total']

In [ ]:
profile.result.frame['corr_kendall']

In [ ]:
profile.result.series['HIC']['acf']